# Stock Analysis using Google Search Analytics

Should I buy or sell a specific stock today based off of how popular it is on google searches?

Group 1 <br>
3 August 2021 <br>
BAIS 6040: Data Programming in Python <br>

## Introduction

## Import

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
import math
import plotly.express as px
import random as rnd
from datetime import date
from datetime import timedelta
from pytrends.request import TrendReq
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, fbeta_score, classification_report
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split    # For generating test/train
from sklearn.linear_model import LinearRegression   # Logistic regression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
%matplotlib inline

## Global Variables and Initialization

In [6]:
dataDir = r"./Data Files/"  #Directory of all data
today = date.today()  # Todays date
rnd.seed(1024)

## Global Functions

In [10]:
# Function gets stock data and trend data if needed
def get_data(ticker):
    if os.path.exists(f"{dataDir}{ticker}_{today}_year.csv"):
        #Get stored data
        stored_data = pd.read_csv(f"{dataDir}{ticker}_{today}_year.csv")
        # Get rid of index name
        stored_data.set_index('Unnamed: 0', inplace=True)
        stored_data.index.name = None
        return stored_data
    else:
        #Get new data
        # Connect to Google API
        pytrends = TrendReq(hl='en-US', tz=360)
        # Set Keyword
        kw_list = [ticker]
        # Google API only shows last 90 days so need to itirate
        # Set start of interval
        date90front = date.today()
        # Initiate dataframe
        trend_data = pd.DataFrame()
        for x in range(4):
            # Set start end of interval
            date90back = date90front - timedelta(days=90)
            # Build Payload of 90 days
            pytrends.build_payload(kw_list,
                                   timeframe=f'{date90back} {date90front}',
                                   geo='')
            trend_90 = pytrends.interest_over_time()
            trend_data = pd.concat([trend_90, trend_data])
            date90front = date90back
        # Get Stock Data
        stock_data = yf.download(ticker,
                                 start=date.today() - timedelta(days=360),
                                 end=date.today(), interval="1d")
        # Combine Data
        new_data = stock_data.join(trend_data)
        
        
        # Rename search interest
        new_data.rename(columns = {ticker: "Search Interest"},inplace = True)
        # Add difference
        new_data["Price Difference"] = new_data["Open"] - new_data["Close"]
        # Add truth value that determines if we want to buy or not that day
        new_data['Buy'] = np.where(new_data['Price Difference'] > 0, 1, 0)
        # Delete isPartial
        del new_data['isPartial']
        # Remove NaN
        new_data.dropna(inplace=True)  
        # Export to data folder
        new_data.to_csv(f"{dataDir}{ticker}_{today}_year.csv")
        return new_data

# Function prints metrics of regression model
def PrintMetricsRegression(test, predictions):
    print(f"Score: {explained_variance_score(test, predictions):.2f}")
    print(f"MAE: {mean_absolute_error(test, predictions):.2f}")
    print(f"RMSE: {math.sqrt(mean_squared_error(test, predictions)):.2f}")
    print(f"r2: {r2_score(test, predictions):.2f}")

# Function prints metrics of classification model
def PrintMetricsClassiffication(test, predictions):
    print("Confusion Matrix:")
    print(confusion_matrix(test, predictions))
    print("------------------")
    print(f"Accuracy: {accuracy_score(test, predictions):.2f}")
    print(f"Recall: {recall_score(test, predictions):.2f}")
    print(f"Prediction: {precision_score(test, predictions):.2f}")
    print(f"f-measure: {fbeta_score(test, predictions, beta=1):.2f}")
    print("------------------")
    print(classification_report(test, predictions))


# Functions gets random data for predictions
def prepareDataForPredictions(X_df):
    numElements = 3
    random_df = []
    for _ in range(numElements):
        dict = {}
        for column in X_df.columns:
            min = 0  # assume min = 0
            maxValue = round(max(X_df[column].values))
            dict[column] = rnd.randint(min, maxValue)
        random_df.append(dict)
    return random_df

# Create categorical dummies
def createCategoricalDummies(dataFrame, categoryList):
    return pd.get_dummies(dataFrame[categoryList], prefix_sep = "::", drop_first = True)

## Stock to Analyze

In [11]:
# Gets Data for last year
StockSearch_df = get_data("GME")
StockSearch_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Search Interest,Price Difference,Buy
2020-07-27,4.020000,4.120000,3.950000,4.010000,4.010000,2472700,19.0,0.010000,1
2020-07-28,3.960000,4.050000,3.920000,3.940000,3.940000,4555400,14.0,0.020000,1
2020-07-29,3.940000,4.180000,3.920000,4.060000,4.060000,2879600,20.0,-0.120000,0
2020-07-30,4.000000,4.230000,3.970000,4.100000,4.100000,2398500,16.0,-0.100000,0
2020-07-31,4.060000,4.160000,3.990000,4.010000,4.010000,1879400,19.0,0.050000,1
...,...,...,...,...,...,...,...,...,...
2021-07-14,180.490005,182.380005,165.070007,167.619995,167.619995,3913800,28.0,12.870010,1
2021-07-15,160.000000,171.990005,158.009995,166.820007,166.820007,4298600,31.0,-6.820007,0
2021-07-16,170.149994,179.470001,166.300003,169.039993,169.039993,3278800,28.0,1.110001,1
2021-07-19,163.300003,176.000000,161.220001,173.490005,173.490005,2436900,28.0,-10.190002,0


## Analytics Model 1 w/ plots

## Analytics Model 2 w/ Plots

## Conclusion

You shoud {buy/sell} with this {##%} confidence